In [1]:
import pandas as pd

In [2]:
#file to load
file_to_load="Resources/purchase_data.csv"

#read file and store into Pandas Data Frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
#display the total number of unique players
total_players = len(purchase_data["SN"].unique())
total_players


576

In [4]:
#display the total number of unique in game items
unique_items = len(purchase_data["Item ID"].unique())
unique_items

179

In [5]:
#calculate average price per item
price_average = purchase_data["Price"].mean()
price_average

3.0509871794871795

In [6]:
#total revenue from all in game purchases
total_revenue=purchase_data["Price"].sum()
total_revenue


2379.77

In [7]:
#total number of purchases 
total_purchases=purchase_data["Purchase ID"].count()
total_purchases

780

In [8]:
#Summary Data Frame/cleaned up for organization 
summary_df_1=pd.DataFrame([{"Number of Unique Items": unique_items,
                           "Average Price": price_average,
                           "Number of Purchases":total_purchases,
                           "Total Revenue":total_revenue}])
summary_df_1

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,3.050987,780,2379.77


In [9]:
#Gender Demographics
gender_data=purchase_data.groupby("Gender")

#used the nunique function to get number of male female and other/non-disclosed players
gender_totals = gender_data["SN"].nunique()
gender_totals

#calculation for gender demo divided by total players
player_percent = gender_totals / total_players * 100

#New Data Frame
gender_totals=pd.DataFrame({"Total Count": gender_totals,"Percentage of Players": player_percent})
gender_totals.head()

#use formatting and use map to add %
gender_totals["Percentage of Players"] = gender_totals["Percentage of Players"].map("{:.2f}%".format)

gender_totals.head()

,Total Count,Percentage of Players
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%


In [10]:
#Purchasing Analysis (Gender)
#calculates the number of items purchases by gender
gender_purchases=purchase_data.groupby(["Gender"]).nunique()["Price"]
gender_purchases

Gender
Female                    79
Male                     144
Other / Non-Disclosed     12
Name: Price, dtype: int64

In [11]:
#calculates average cost of item bough per gender 
avg_cost_gender= purchase_data.groupby(["Gender"]).mean()["Price"]
avg_cost_gender


Gender
Female                   3.203009
Male                     3.017853
Other / Non-Disclosed    3.346000
Name: Price, dtype: float64

In [12]:
#calculates the total of the purchases by gender
g_purchases = purchase_data.groupby(["Gender"]).sum()["Price"]
g_purchases

Gender
Female                    361.94
Male                     1967.64
Other / Non-Disclosed      50.19
Name: Price, dtype: float64

In [13]:
gender_total_total =purchase_data["Gender"].value_counts()

#calculates the average totals per person/per gender
average_totals_pers = g_purchases / gender_total_total

In [14]:
#Gender Summary DF/cleaned up for organizationg
gender_summary=pd.DataFrame({"Purchase Count": gender_purchases,
                           "Average Purchase Price": avg_cost_gender,
                           "Total Purchase Value":g_purchases,
                           "Avg Total Purchase Per Person":average_totals_pers})

#use map to format and add $
gender_summary["Average Purchase Price"] = gender_summary["Average Purchase Price"].map("${:.2f}".format)
gender_summary["Total Purchase Value"] = gender_summary["Total Purchase Value"].map("${:.2f}".format)
gender_summary["Avg Total Purchase Per Person"] = gender_summary["Avg Total Purchase Per Person"].map("${:.2f}".format)

gender_summary.head()

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase Per Person
Gender,,,,
Female,79,$3.20,$361.94,$3.20
Male,144,$3.02,$1967.64,$3.02
Other / Non-Disclosed,12,$3.35,$50.19,$3.35


In [32]:
#Age Demographics

#Find the min and max ages of the players to aid in creating bins
print(purchase_data["Age"].max())
print(purchase_data["Age"].min())


45
7


In [38]:
#create bins and labels
bins= [0,9,14,19,24,29,34,39,100]
age_groups=["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

#
age_stats = purchase_data.loc[:,["Age","SN"]]
age_stats = age_stats.drop_duplicates()
age_players= age_stats["Age"].value_counts()

#Slice the data and place it into the bins
age_data = pd.cut(age_stats["Age"], bins, labels=age_groups).value_counts()

In [39]:
#Calculation to find percentage of players by age
age_percent = age_data / total_players * 100

In [37]:
#Summary
age_summary_2 = pd.DataFrame({"Total Count": age_data,
                              "Percentage of Players": age_percent})

#Formatting and use map to add decimal and %
age_summary_2["Percentage of Players"] = age_summary_2["Percentage of Players"].map("{:.2f}%".format)

# Sort and show
age_summary_2 = age_summary_2.sort_index()
age_summary_2



,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [45]:
#Purchasing Analysis by age 

#place existing playerbase into bins
purchase_data["Age Ranges"] = pd.cut(purchase_data["Age"], bins, labels=age_groups)
age_purchases=purchase_data["Age Ranges"].value_counts()

# Calculate average purchase price by age
avg_price_age = purchase_data.groupby("Age Ranges")["Price"].mean()

# Calculate total purchase by ages
total_purch_age = purchase_data.groupby("Age Ranges")["Price"].sum()

# Calculate average total purchase per player
avg_purch_age = total_purch_age / age_data

# Summary
age_summary_3= pd.DataFrame({"Purchase Count" : age_purchases,
                                     "Average Purchase Price" : avg_price_age,
                                     "Total Purchase Value" : total_purch_age,
                                     "Avg Total Purchase per Person" : avg_purch_age})


# Formatting, use map to add $
age_summary_3["Average Purchase Price"] = age_summary_3["Average Purchase Price"].map("${:.2f}".format)
age_summary_3["Total Purchase Value"] = age_summary_3["Total Purchase Value"].map("${:.2f}".format)
age_summary_3["Avg Total Purchase per Person"] = age_summary_3["Avg Total Purchase per Person"].map("${:.2f}".format)

age_summary_3


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [48]:
#Top Spenders

# Calculate purchase count by player
top_purch_count = purchase_data.groupby(["SN"]).count()["Price"]
top_purch_count

SN
Adairialis76     1
Adastirin33      1
Aeda94           1
Aela59           1
Aelaria33        1
                ..
Yathecal82       3
Yathedeu43       2
Yoishirrala98    1
Zhisrisu83       2
Zontibe81        3
Name: Price, Length: 576, dtype: int64

In [49]:
# Calculate average purchase price per player
top_avg_cost= purchase_data.groupby(["SN"]).mean()["Price"]
top_avg_cost

SN
Adairialis76     2.280000
Adastirin33      4.480000
Aeda94           4.910000
Aela59           4.320000
Aelaria33        1.790000
                   ...   
Yathecal82       2.073333
Yathedeu43       3.010000
Yoishirrala98    4.580000
Zhisrisu83       3.945000
Zontibe81        2.676667
Name: Price, Length: 576, dtype: float64

In [50]:
# Calculate total purchase value sum
top_total_purch = purchase_data.groupby(["SN"]).sum()["Price"]
top_total_purch




SN
Adairialis76     2.28
Adastirin33      4.48
Aeda94           4.91
Aela59           4.32
Aelaria33        1.79
                 ... 
Yathecal82       6.22
Yathedeu43       6.02
Yoishirrala98    4.58
Zhisrisu83       7.89
Zontibe81        8.03
Name: Price, Length: 576, dtype: float64

In [61]:
#Summary
spender_summary= pd.DataFrame({"Purchase Count" : top_purch_count,
                                     "Average Purchase Price" : top_avg_cost,
                                     "Total Purchase Value" : top_total_purch})


#Formatting
spender_summary["Average Purchase Price"] = spender_summary["Average Purchase Price"].map("${:.2f}".format)
spender_summary["Total Purchase Value"] = spender_summary["Total Purchase Value"].map("${:.2f}".format)

spender_summary = spender_summary.sort_values(by = "Purchase Count",ascending = False)
spender_summary.head()



,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Iral74,4,$3.40,$13.62
Idastidru52,4,$3.86,$15.45
Asur53,3,$2.48,$7.44
Inguron55,3,$3.70,$11.11


In [72]:
#Most Popular Items
#Retrieve the Item ID, Item Name, and Item Price columns, group by Item ID
item_purch= purchase_data.groupby(["Item ID","Item Name"])["Price"].count()

#calculate for 
item_pop = purchase_data.groupby(["Item ID","Item Name"])["Price"].sum()

# Retrive the item price
ind_item = popular_item / item_purch

# Create a summary
item_summary = pd.DataFrame({"Purchase Count": item_purch,
                             "Item Price" : ind_item,
                             "Total Purchase Value": item_pop})


# Formatting, use map to add $
item_summary["Item Price"] = item_summary["Item Price"].map("${:.2f}".format)
item_summary["Total Purchase Value"] = item_summary["Total Purchase Value"].map("${:.2f}".format)


item_summary = item_summary.sort_values(by = "Purchase Count",ascending = False)
item_summary.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
